In [1]:
import numpy as np
import pickle
import random
import torch

In [2]:
def insert_dataset_generation():
    joint_trajs = np.load('data/insert_records/online/sequence/expert_joints_poses_new.npy', allow_pickle=True)
    insert_goals = np.load('data/insert_records/online/sequence/expert_targets.npy', allow_pickle=True)
    insert_len = 5

    d4rl_format_trajs = []
    d4rl_format_path = 'data/insert-expert.pkl'

    for jid, jt in enumerate(joint_trajs):
        observations = []
        next_observations = []
        actions = []
        rewards = []
        terminals = []
        goals = []

        sample_ratio = int(len(jt)/insert_len)
        jt = jt[::sample_ratio]
        if len(jt)>insert_len:
            print('remove')
            jt = np.delete(jt,obj=-2, axis=0)

        for tid, item in enumerate(jt[:-1]):
            observations.append(item)
            actions.append(jt[tid+1] )#- item)
            next_observations.append(jt[tid+1])
            rewards.append(0)
            goals.append(insert_goals[jid])
            print(insert_goals[jid])
            
            if tid!=len(jt[:-1])-1:
                terminals.append(False)
            else:
                terminals.append(True)
        
        for __ in range(5):
            observations.append(observations[-1])
            actions.append(actions[-1] )#- item)
            next_observations.append(next_observations[-1])
            rewards.append(0)
            goals.append(goals[-1])
            print(insert_goals[jid])
            
            terminals.append(terminals[-1])
        
        observations = np.array(observations)
        actions = np.array(actions)
        next_observations = np.array(next_observations)
        rewards = np.array(rewards)
        goals = np.array(goals)
        terminals = np.array(terminals)
        
        d4rl_traj = {'observations':observations,
                    'next_observations':next_observations,
                    'actions':actions,
                    'rewards':rewards,
                    'terminals':terminals,
                    'goals': goals}

        d4rl_format_trajs.append(d4rl_traj)
        

    with open(d4rl_format_path, 'wb') as fp:
        pickle.dump(d4rl_format_trajs, fp)  

In [3]:
insert_dataset_generation()

remove
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
[ 6.09441209e-01 -2.56966434e-02 -1.35051242e+02]
remove
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.47303711e+02]
[ 6.25874870e-01 -4.05697019e-02 -1.

In [26]:
#with open('data/hopper-expert-v2.pkl', 'rb') as f:
#        mujoco_trajs = pickle.load(f)
        
with open('data/insert-expert.pkl', 'rb') as f:
        insert_trajs = pickle.load(f)
        


[array([[ 5.54495004e-01, -4.45091403e-02,  1.85578956e-01,
         -2.43707943e+00, -7.41834547e-02, -3.20938018e-01,
          3.71275034e-03, -1.80215480e-03, -5.79447191e-05,
          3.47841575e-03,  1.66343279e-03, -7.17873823e-03,
          6.08430353e-01, -8.37070946e-01,  2.65544724e+00,
          3.58585526e-01, -1.29748937e+00, -2.63291101e-01],
        [ 5.58208896e-01, -4.63116001e-02,  1.85520209e-01,
         -2.43359965e+00, -7.25147678e-02, -3.28110905e-01,
          1.31951593e-03, -9.27169433e-04, -8.59846561e-04,
         -3.27503988e-03,  5.75508331e-04, -2.02613828e-03,
          8.09983284e-01, -6.29903842e-01,  2.57383879e+00,
          2.02935244e-01, -1.24900689e+00, -3.04806599e-01],
        [ 5.59529741e-01, -4.72349602e-02,  1.84660821e-01,
         -2.43687746e+00, -7.19414673e-02, -3.30138797e-01,
         -7.13370384e-04, -6.35847601e-03, -3.84147552e-04,
         -7.99578621e-02, -2.22963443e-02,  6.04614151e-03,
         -1.69960529e-01, -1.40911452e

In [95]:
len(mujoco_trajs)
insert_trajs[0]['goals'][0].shape[0]


3

In [96]:
mujoco_trajs[0]


state_dim = mujoco_trajs[0]['observations'][0].shape[0]
act_dim = mujoco_trajs[0]['actions'][0].shape[0]
state_dim, act_dim

states, traj_lens, returns = [], [], []
for path in mujoco_trajs:
    states.append(path['observations'])
    traj_lens.append(len(path['observations']))
    returns.append(path['rewards'].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6

num_timesteps = sum(traj_lens)

traj_lens, len(returns)

(array([1000, 1000, 1000, ..., 1000,  452, 1000]), 1027)

In [97]:
pct_traj = 1

# only train on top pct_traj trajectories (for %BC experiment)
num_timesteps = max(int(pct_traj*num_timesteps), 1)
sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
print(timesteps)

ind = len(mujoco_trajs) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]

# used to reweight sampling so we sample according to timesteps instead of trajectories
p_sample = traj_lens[sorted_inds] / sum(traj_lens[sorted_inds])

num_timesteps, p_sample, sorted_inds, 

1000


(999494,
 array([0.00045544, 0.00044943, 0.00045244, ..., 0.00100096, 0.00100096,
        0.00100096]),
 array([  42,  548, 1025, ...,  847,  649,  385]))

In [98]:
def discount_cumsum(x, gamma):
    discount_cumsum = np.zeros_like(x)
    discount_cumsum[-1] = x[-1]
    for t in reversed(range(x.shape[0]-1)):
        discount_cumsum[t] = x[t] + gamma * discount_cumsum[t+1]
    return discount_cumsum

scale = 1000. 
K=20
batch_size=2
max_len=K
max_ep_len = 1000
device = 'cpu'
#def get_batch(batch_size=256, max_len=K):
batch_inds = np.random.choice(
    np.arange(num_trajectories),
    size=batch_size,
    replace=True,
    p=p_sample,  # reweights so we sample according to timesteps
)

s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
for i in range(batch_size):
    traj = mujoco_trajs[int(sorted_inds[batch_inds[i]])]
    #print(traj)
    #a()
    si = random.randint(0, traj['rewards'].shape[0] - 1)
    
    

    # get sequences from dataset, segment a sub-traj from traj, horizon as max_len
    s.append(traj['observations'][si:si + max_len].reshape(1, -1, state_dim))
    a.append(traj['actions'][si:si + max_len].reshape(1, -1, act_dim))
    r.append(traj['rewards'][si:si + max_len].reshape(1, -1, 1))
    if 'terminals' in traj:
        d.append(traj['terminals'][si:si + max_len].reshape(1, -1))
    else:
        d.append(traj['dones'][si:si + max_len].reshape(1, -1))
    
    #print(np.arange(si, si + s[-1].shape[1]))
    #a()
    # get the time-stamp index for the sub-traj, e.g. [901,902, ..., 920]
    timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
    timesteps[-1][timesteps[-1] >= max_ep_len] = max_ep_len-1  # padding cutoff
    
    # get the RTG items, e.g. [2049, 2047, 2040, ..., 1990],then reshape to dim=[1,1,1]
    
    print(discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1).shape)
    print(traj['observations'][si:si + max_len].reshape(1, -1, state_dim).shape)
    a()
    rtg.append(discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1))
    test_goal_s = traj['observations'][0]
    test_goal_s = np.broadcast_to(test_goal_s,(1,max_len+1,11))
    
    #print(discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1))
    #a()
    print(rtg[-1].shape)
    if rtg[-1].shape[1] <= s[-1].shape[1]:
        rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)
        print(rtg)
    
    if rtg[-1].shape[1] <= s[-1].shape[1]:
        rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)
        print(rtg)

    # padding and state + reward normalization
    tlen = s[-1].shape[1]

    s[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), s[-1]], axis=1)

    s[-1] = (s[-1] - state_mean) / state_std
    tmp = a[-1]
    a[-1] = np.concatenate([np.ones((1, max_len - tlen, act_dim)) * -10., a[-1]], axis=1)
    #print(tmp - a[-1])
    r[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), r[-1]], axis=1)
    d[-1] = np.concatenate([np.ones((1, max_len - tlen)) * 2, d[-1]], axis=1)
    rtg[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), rtg[-1]], axis=1) / scale
    timesteps[-1] = np.concatenate([np.zeros((1, max_len - tlen)), timesteps[-1]], axis=1)
    mask.append(np.concatenate([np.zeros((1, max_len - tlen)), np.ones((1, tlen))], axis=1))

s = torch.from_numpy(np.concatenate(s, axis=0)).to(dtype=torch.float32, device=device)
a = torch.from_numpy(np.concatenate(a, axis=0)).to(dtype=torch.float32, device=device)
r = torch.from_numpy(np.concatenate(r, axis=0)).to(dtype=torch.float32, device=device)
d = torch.from_numpy(np.concatenate(d, axis=0)).to(dtype=torch.long, device=device)
rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).to(dtype=torch.float32, device=device)
timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).to(dtype=torch.long, device=device)
mask = torch.from_numpy(np.concatenate(mask, axis=0)).to(device=device)

print(rtg.shape, rtg)
'''
#print(s.shape, s)
print(a.shape, a)
#print(r.shape, r)
#print(d.shape, d)
#print(rtg.shape, rtg)
print(timesteps.shape, timesteps)
print(mask.shape, mask)

print(rtg.shape, s.shape)
'''
#return s, a, r, d, rtg, timesteps, mask
#states, actions, rewards, dones, rtg, timesteps, attention_mask = self.get_batch(self.batch_size)

(1, 21, 1)
(1, 20, 11)


TypeError: 'list' object is not callable

In [113]:
device = 'cuda'
log_to_wandb = False
env_name = 'insert'
model_type = 'dt'

# load dataset
dataset_path = f'/home/wenyan/00-AI/decision-transformer/insert/data/insert-expert.pkl' #f'data/{env_name}-{dataset}-v2.pkl'
with open(dataset_path, 'rb') as f:
    trajectories = pickle.load(f)
    state_dim = trajectories[0]['observations'][0].shape[0]
    act_dim = trajectories[0]['actions'][0].shape[0]
    goal_dim = trajectories[0]['goals'][0].shape[0]

if env_name == 'hopper':
    env = gym.make('Hopper-v3')
    max_ep_len = 1000
    env_targets = [3600, 1800]  # evaluation conditioning targets
    scale = 1000.  # normalization for rewards/returns
elif env_name == 'insert':
    max_ep_len = 5
    scale = 10.
else:
    raise NotImplementedError

if model_type == 'bc':
    env_targets = env_targets[:1]  # since BC ignores target, no need for different evaluations


# save all path information into separate lists
mode =  'normal'
states, traj_lens, returns, goals = [], [], [], []
for path in trajectories:
    if mode == 'delayed':  # delayed: all rewards moved to end of trajectory
        path['rewards'][-1] = path['rewards'].sum()
        path['rewards'][:-1] = 0.
    states.append(path['observations'])
    traj_lens.append(len(path['observations']))
    returns.append(0)
    goals.append(path['goals'])

#traj_lens, returns = np.array(traj_lens), np.array(returns)
traj_lens = np.array(traj_lens)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6

# used for goal normalization
goals = np.concatenate(goals, axis=0)
goal_mean, goal_std = np.mean(goals, axis=0), np.std(goals, axis=0) + 1e-6

num_timesteps = sum(traj_lens)

print('=' * 50)
#print(f'Starting new experiment: {env_name} {dataset}')
print(f'{len(traj_lens)} trajectories, {num_timesteps} timesteps found')
#print(f'Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}')
#print(f'Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}')
print('=' * 50)

K= 5
batch_size = 1#variant['batch_size']
num_eval_episodes = 1#variant['num_eval_episodes']
pct_traj = 1#variant.get('pct_traj', 1.)

# only train on top pct_traj trajectories (for %BC experiment)
num_timesteps = max(int(pct_traj*num_timesteps), 1)
sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]

# used to reweight sampling so we sample according to timesteps instead of trajectories
p_sample = traj_lens[sorted_inds] / sum(traj_lens[sorted_inds])


batch_size=256
max_len=K
batch_inds = np.random.choice(
    np.arange(num_trajectories),
    size=batch_size,
    replace=True,
    #p=p_sample,  # reweights so we sample according to timesteps
)

s, a, r, d, rtg, timesteps, mask, gs = [], [], [], [], [], [], [], []
for i in range(batch_size):
    traj = trajectories[int(sorted_inds[batch_inds[i]])]
    si = random.randint(0, traj['goals'].shape[0] - 1)

    # get sequences from dataset
    s.append(traj['observations'][si:si + max_len].reshape(1, -1, state_dim))
    a.append(traj['actions'][si:si + max_len].reshape(1, -1, act_dim))
    r.append(traj['rewards'][si:si + max_len].reshape(1, -1, 1))
    if 'terminals' in traj:
        d.append(traj['terminals'][si:si + max_len].reshape(1, -1))
    else:
        d.append(traj['dones'][si:si + max_len].reshape(1, -1))
    timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
    timesteps[-1][timesteps[-1] >= max_ep_len] = max_ep_len-1  # padding cutoff
    #rtg.append(discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1))
    #discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1)
    #print(discount_cumsum(traj['rewards'][si:], gamma=1.))
    #s()
    goal = traj['goals'][0]
    goal = np.broadcast_to(goal,(s[-1].shape[0],s[-1].shape[1],goal_dim))
    gs.append(goal)

    #if rtg[-1].shape[1] <= s[-1].shape[1]:
    #    rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

    if gs[-1].shape[1] <= s[-1].shape[1]:
        gs[-1] = np.concatenate([gs[-1], np.zeros((1, 1, goal_dim))], axis=1)
        #print(gs)

    # padding and state + reward normalization
    tlen = s[-1].shape[1]
    
    s[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), s[-1]], axis=1)
    s[-1] = (s[-1] - state_mean) / state_std
    a[-1] = np.concatenate([np.ones((1, max_len - tlen, act_dim)) * -10., a[-1]], axis=1)
    r[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), r[-1]], axis=1)
    d[-1] = np.concatenate([np.ones((1, max_len - tlen)) * 2, d[-1]], axis=1)
    #rtg[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), rtg[-1]], axis=1) / scale
    timesteps[-1] = np.concatenate([np.zeros((1, max_len - tlen)), timesteps[-1]], axis=1)
    mask.append(np.concatenate([np.zeros((1, max_len - tlen)), np.ones((1, tlen))], axis=1))

    gs[-1] =  np.concatenate([np.zeros((1, max_len - tlen, goal_dim)), gs[-1]], axis=1)
    gs[-1] =  (gs[-1] - goal_mean) / goal_std
    
    #print(s[-1].shape, gs[-1].shape)

s = torch.from_numpy(np.concatenate(s, axis=0)).to(dtype=torch.float32, device=device)
a = torch.from_numpy(np.concatenate(a, axis=0)).to(dtype=torch.float32, device=device)
r = torch.from_numpy(np.concatenate(r, axis=0)).to(dtype=torch.float32, device=device)
d = torch.from_numpy(np.concatenate(d, axis=0)).to(dtype=torch.long, device=device)
#rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).to(dtype=torch.float32, device=device)
timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).to(dtype=torch.long, device=device)
mask = torch.from_numpy(np.concatenate(mask, axis=0)).to(device=device)

#print(gs)
gs = torch.from_numpy(np.concatenate(gs, axis=0)).to(dtype=torch.float32, device=device)

for trj in trajectories:
    print(trj['actions'])
a[-1][1]

54 trajectories, 270 timesteps found
[[-2.28287650e-02  1.55754372e-02 -5.25894606e-03  2.02451130e-02
  -5.22713317e-05 -2.81608370e-04  8.91862613e-02]
 [ 4.32595559e-02  2.33422490e-03  9.40192586e-03  3.87318958e-03
  -8.32850910e-04  3.67423164e-04 -2.54232183e-01]
 [ 3.84648657e-03  1.86892590e-03 -2.98914534e-04  7.70602886e-03
  -4.48607858e-04 -6.62034072e-03  1.04016377e-01]
 [ 3.02894341e-03  5.84766197e-03 -1.24026895e-03  1.05010094e-02
   2.75784473e-04 -7.43371279e-03  3.31628465e-02]
 [-1.01744239e-02  7.91118859e-02 -5.62955768e-03  2.41010669e-01
  -3.59700330e-04 -4.25222120e-01 -5.39796418e-02]]
[[ 1.81247851e-02  3.51645692e-02  2.30909574e-02  5.41746253e-02
  -5.41909759e-04 -1.48575125e-04 -6.17010207e-02]
 [-4.24857813e-03  1.10703017e-02 -6.36150923e-02 -5.37742434e-03
  -1.56495177e-03 -2.34733944e-03  1.55452981e-01]
 [ 2.01482293e-02 -1.24599788e-03  2.46940991e-02  6.61633552e-03
   3.45512785e-04  2.53477362e-03 -5.80346918e-02]
 [-8.71709857e-03  3.65709

tensor([ 0.0023,  0.0108,  0.0018,  0.0191, -0.0003,  0.0055, -0.1937],
       device='cuda:0')